<a href="https://colab.research.google.com/github/EstebanAgudeloR/Proyecto-8-API-GATEWAY/blob/main/Api_Gateway_Codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import boto3
import time
import uuid

#Este script recibe la petición del API Gateway, procesa los datos y los guarda en DynamoDB.
# Conectar con DynamoDB
dynamodb = boto3.resource('dynamodb')
# ASEGÚRATE DE QUE ESTE NOMBRE SEA EXACTO AL QUE PUSISTE EN EL PASO 1
table = dynamodb.Table('Proyecto8-Visitas')

def lambda_handler(event, context):

    # 1. Preparar los datos a guardar
    # Generamos un ID único y la hora actual
    item_id = str(uuid.uuid4())
    timestamp = str(time.time())

    # 2. Intentar guardar en DynamoDB
    try:
        table.put_item(
            Item={
                'id': item_id,
                'mensaje': 'Hola desde la API',
                'timestamp': timestamp,
                'status': 'visitante_registrado'
            }
        )
        mensaje_resultado = "Éxito: Datos guardados en DynamoDB"
    except Exception as e:
        print(e)
        mensaje_resultado = f"Error guardando en DB: {str(e)}"

    # 3. Responder al usuario (Formato Proxy Integration)
    return {
        "statusCode": 200,
        "headers": {
            "Content-Type": "application/json",
            "Access-Control-Allow-Origin": "*" # CORS simple
        },
        "body": json.dumps({
            "mensaje": "Hola desde Lambda + DynamoDB",
            "db_status": mensaje_resultado,
            "id_generado": item_id
        })
    }



    #Simulacion de la instncia EC2 con lambda de manera sencilla
    import json

def lambda_handler(event, context):
    # [cite_start]La respuesta esperada del servidor Apache era "Hola desde EC2"[cite: 142].
    # Configuramos el encabezado para simular que es una respuesta HTML.
    return {
        "statusCode": 200,
        "headers": {"Content-Type": "text/html"},
        "body": "Hola desde EC2"
    }









In [ ]:

#Script en Python para demostrar cómo un cliente debe enviar el Token y la API Key obligatoriamente.
 import requests
# CONFIGURACIÓN
API_URL = "https://tu-api-id.execute-api.us-east-1.amazonaws.com/v1"
API_KEY = "tu_api_key_aqui"
JWT_TOKEN = "tu_token_jwt_de_cognito_aqui"

def test_secure_endpoint():
    """
    Prueba de integración con seguridad completa.
    Demuestra el paso por WAF -> API Gateway -> Lambda.
    """
    headers = {
        'x-api-key': API_KEY,             # Requerido por Usage Plan
        'Authorization': f'Bearer {JWT_TOKEN}', # Requerido por Cognito
        'Content-Type': 'application/json'
    }

    print(f"Enviando petición a: {API_URL}/lambda")

    try:
        response = requests.get(f"{API_URL}/lambda", headers=headers)

        print(f"Estado HTTP: {response.status_code}")
        print("Respuesta del servidor:")
        print(response.json())

        if response.status_code == 200:
            print("\n PRUEBA EXITOSA: Autenticación y Autorización correctas.")
        else:
            print("\n FALLO: Revisa WAF, Token o API Key.")

    except Exception as e:
        print(f"Error de conexión: {e}")

if __name__ == "__main__":
    test_secure_endpoint()